In [9]:
from l2r_class import *

In [10]:
import json
fichier = 'msmarco-hard-negatives.jsonl'
with open(fichier, 'r') as f:
    first_line = f.readline()
    data = json.loads(first_line)
    print(data['neg']['bm25'])

[{'pid': '6948601', 'bm25-score': 26.068258, 'ce-score': 3.581289291381836}, {'pid': '5129919', 'bm25-score': 26.007563, 'ce-score': 8.257364273071289}, {'pid': '6717931', 'bm25-score': 25.934547, 'ce-score': 8.866464614868164}, {'pid': '1065943', 'bm25-score': 25.258688, 'ce-score': 5.258519172668457}, {'pid': '1626276', 'bm25-score': 25.153275, 'ce-score': 4.19371223449707}, {'pid': '981824', 'bm25-score': 24.716213, 'ce-score': 8.563857078552246}, {'pid': '6449111', 'bm25-score': 24.62115, 'ce-score': 4.907355785369873}, {'pid': '1028927', 'bm25-score': 24.54112, 'ce-score': 7.617893695831299}, {'pid': '2524942', 'bm25-score': 24.355902, 'ce-score': 1.5268436670303345}, {'pid': '5810175', 'bm25-score': 24.315733, 'ce-score': -0.6152520179748535}, {'pid': '6236527', 'bm25-score': 24.3093, 'ce-score': -2.9456772804260254}, {'pid': '7179545', 'bm25-score': 24.130537, 'ce-score': 10.018341064453125}, {'pid': '168979', 'bm25-score': 24.130537, 'ce-score': 10.202350616455078}, {'pid': '15

In [11]:
import json

# Load the JSON data from a file
data_list = []
with open('msmarco-hard-negatives.jsonl', 'r') as file:
    for line in file:
        #print(json.loads(line))
        data_list.append(json.loads(line))
        if len(data_list) > 50:
            break

# Now data_list contains multiple JSON objects, one per line from the file
data_id = []

for data in data_list:
    # Extract positive examples
    pos_examples = [(data['qid'], pos['pid'], 1) for i, pos in enumerate(data['pos']) if i < 1]

    # Extract negative examples with a bm25_score
    neg_examples = [(data['qid'], neg['pid'], 0) for i, neg in enumerate(data['neg']['bm25']) if i < 9]

    # Combine positive and negative examples
    data_id.extend(pos_examples + neg_examples)

# Print the final_data
print(len(data_id))
print(data_id[:20])

510
[('571018', '7349777', 1), ('571018', '6948601', 0), ('571018', '5129919', 0), ('571018', '6717931', 0), ('571018', '1065943', 0), ('571018', '1626276', 0), ('571018', '981824', 0), ('571018', '6449111', 0), ('571018', '1028927', 0), ('571018', '2524942', 0), ('832106', '5090093', 1), ('832106', '5090096', 0), ('832106', '210166', 0), ('832106', '4877671', 0), ('832106', '851808', 0), ('832106', '210158', 0), ('832106', '2492227', 0), ('832106', '3094439', 0), ('832106', '5434531', 0), ('832106', '210165', 0)]


In [12]:
import ir_datasets

# Load the msmarco dataset
dataset = ir_datasets.load('msmarco-passage/train')
store = dataset.docs_store()
print(dataset)

Dataset(id='msmarco-passage/train', provides=['docs', 'queries', 'qrels', 'scoreddocs', 'docpairs'])


In [13]:
dict_query = {}

for query in dataset.queries_iter():
    dict_query[query.query_id] = query.text
    

In [14]:
import numpy as np

# Create an empty list to store the modified data
final_data = []

for data in data_id:
    # Extract the values from the original data
    dict_value = dict_query[data[0]]
    store_value = store.get(data[1])[1]
    class_value = data[2]
    
    # Perform one-hot encoding
    class_vector = np.zeros(2)
    class_vector[class_value] = 1
    
    # Create a modified tuple with the encoded class vector
    modified_tuple = (dict_value, store_value, class_vector)
    
    # Append the modified tuple to the list
    final_data.append(modified_tuple)

# Print the modified_data
print(len(final_data))

510


In [15]:
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
bert_model = BertModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'fit_denses.0.weight', 'fit_denses.4.bias', 'cls.seq_relationship.weight', 'fit_denses.4.weight', 'fit_denses.3.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'fit_denses.2.bias', 'fit_denses.2.weight', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.1.bias', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.0.bias', 'cls.predictions.bias', 'fit_denses.3.bias', 'fit_denses.1.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

### Background: Label Smoothing (LS)

In [8]:
dataset = LTRDataset(final_data, tokenizer, max_len=512)

# Create data loaders
loader = DataLoader(dataset, batch_size=16, num_workers=4)

# Instantiate the model
model = BertLTRModel(bert_model)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=1e-8)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in loader:
        # Move batch tensors to the same device as the model 
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        relevance = batch['relevance'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Compute loss
        loss = criterion(outputs, relevance) 

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch}, Loss: {total_loss/len(loader)}')

/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (de

Epoch 0, Loss: 0.6332044396549463


### T-LS

In [16]:
def learnModel(final_data, num_epochs=1, tls=-1, eps=1e-8):
	dataset = LTRDataset(final_data, tokenizer, max_len=512)

	# Create data loaders
	loader = DataLoader(dataset, batch_size=16, num_workers=4)

	# Instantiate the model
	model = BertLTRModel(bert_model)

	# Move model to GPU if available
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	model = model.to(device)

	# Define loss function and optimizer
	criterion = nn.CrossEntropyLoss(label_smoothing=eps)
	optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

	# Training loop
	for epoch in range(num_epochs):
		model.train()
		total_loss = 0

		for batch in loader:
			# Move batch tensors to the same device as the model 
			input_ids = batch['input_ids'].to(device)
			attention_mask = batch['attention_mask'].to(device)
			relevance = batch['relevance'].to(device)

			# Forward pass
			outputs = model(input_ids=input_ids, attention_mask=attention_mask)

			# Compute loss
			if tls == num_epochs:
				criterion = nn.CrossEntropyLoss()
			loss = criterion(outputs, relevance) 

			# Backward pass and optimize
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			total_loss += loss.item()
		print(f'Epoch {epoch}, Loss: {total_loss/len(loader)}')
	

In [17]:
learnModel(final_data)

/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (de

Epoch 0, Loss: 0.5826489627361298
